In [6]:
import enum
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from gym.spaces import Discrete, Tuple
import pickle
import pandas as pd
from random import randint, getrandbits
import datetime
from datetime import timedelta
import os
from datetime import date
import copy
import math, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named 'torch.nn'

In [ ]:
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, num_actions*2)
        )
        self.num_actions = num_actions
        
    def forward(self, x):
        return self.layers(x)


ImportError: No module named 'torch.nn'

In [ ]:
def get_action(env, state, net, epsilon=0.0):
    if random.random() > epsilon:
        state = torch.autograd.Variable(torch.FloatTensor(state).unsqueeze(0)) # adds extra dim at front
        q_value = net(state)
        action = np.array([
            1 if q_value[:,a*2]<q_value[:,a*2+1] else 0
            for a in range(env.action_space.n)
        ], dtype=np.uint8)
    else:
        action = env.action_space.sample()
    return action


In [ ]:
class HUFSEnv_v1(gym.Env):
    def __init__(self, start_date):
        dataset = 'DataSet1'
        df_inventory = pickle.load(open(f'{dataset}/inventory_df.dump', 'rb')) #column = ["arrival_time","VDCi","VDCj..."]
        df_from_3A = pickle.load(open(f'{dataset}/df_from_3A.dump', 'rb')) #column = ["arrival_time","num_car_transffered"]
        vdc_capacity_dic = pickle.load(open(f'{dataset}/vdc_capacity_dic.dump', 'rb'))
        distance_matrix = pickle.load(open(f'{dataset}/dist_mat_dataset_1.dump', 'rb'))
        location_index_dic = pickle.load(open(f'{dataset}/location_index_dic_dataset_1.dump', 'rb'))
        vdc_rail_dic = pickle.load(open(f'{dataset}/vdc_rail_dic.dump', 'rb'))
        VDC = list(vdc_capacity_dic.keys())
        start_date = pd.to_datetime(start_date)
        DECISION_INTERVAL = 6

        Link_info = {
            i: {
                j: {
                    'distance': distance_matrix[location_index_dic[i]][location_index_dic[j]],
                    'rail': vdc_rail_dic[i] and vdc_rail_dic[j]
                }
                for j in VDC
                if i != j
            }
            for i in VDC
        }
        for i in VDC:
            for j in VDC:
                if i != j:
                    Link_info[i][j]['time'] = Link_info[i][j]['distance'] / (10.0 if Link_info[i][j]['rail'] else 30.0)

        self.Link_info = Link_info
        self.vdc_rail_dic = pickle.load(open(f'{dataset}/vdc_rail_dic.dump', 'rb'))
        self.DECISION_INTERVAL = DECISION_INTERVAL
        self.TRANING_STEPS = 2000
        self.MIN_DWELL_TIME = 2
        self.NUM_LOOKING_FORWARD = 48 // self.DECISION_INTERVAL
        self.initial_data ={
            "df_inventory":copy.deepcopy(df_inventory),
            "start_date": start_date
        }
        self.df_from_3A = df_from_3A
        self.action_space = spaces.MultiBinary(1)
        self.action_space.dtype = np.uint8
        self.observation_space = spaces.Box(low=0, high=1000, shape=(self.NUM_LOOKING_FORWARD * 2, 1), dtype=np.uint16)
        self.VDC_list_without_3A = list(vdc_capacity_dic.keys())
        self.VDC_list_without_3A.remove("3A")
        self.rdly_selected_vdc=random.choice(self.VDC_list_without_3A)
    
    def get_state(self):
        inventory_state = [
            self.next_vdc_capa - sum(self.selected_vdc_inventory_df[self.selected_vdc_inventory_df["arrival_time"] 
                                                                    <= self.current_time + timedelta(days=0.85 * t)]
                                     [self.rdly_selected_vdc])
            for t in range(self.NUM_LOOKING_FORWARD)
        ]
        car_state = [
            sum(self.selected_vdc_from_3A[self.selected_vdc_from_3A["arrival_time"] <= 
                                           self.current_time - timedelta(days=self.MIN_DWELL_TIME) + 
                                           timedelta(hours=t*self.DECISION_INTERVAL)][self.rdly_selected_vdc]) 
            - self.num_of_send
            for t in range(self.NUM_LOOKING_FORWARD)
        ]
        car_state = np.array(car_state)
        q = car_state[0]//20
        car_state[0] = car_state[0]%20
        car_state[1:] -= (20*q)
        car_state=list(car_state)        
        return car_state + inventory_state, q

    def step(self, action):
        trans_reward = 0
        lead_time_reward = 0
        overflow_cost = 0        
        state = self.state
        num_cars = state[0]        
        from_vdc = self.me
        to_vdc = self.rdly_selected_vdc
        num_car_send_t0=0
        
        if self.steps == self.TRANING_STEPS:
            trans_reward = self.calc_transport_cost(from_vdc,to_vdc,num_cars)
            trans_reward += self.calc_transport_cost2(from_vdc,to_vdc,num_cars)
        else:
            if action == 1:  
                self.num_of_send += num_cars
                num_car_send_t0 += num_cars
                trans_reward = self.calc_transport_cost(from_vdc,to_vdc,num_cars)
                ##.. update inventory DataFrame.
                link_info = self.Link_info[from_vdc][to_vdc]
                delivery_time = link_info["time"]
                dt = self.current_time + timedelta(hours = delivery_time) + timedelta(days = 1)
                delivered_time = dt.strftime("%Y-%m-%d %H:00:00")
                idx = self.selected_vdc_inventory_df.index[self.selected_vdc_inventory_df["arrival_time"]==delivered_time][0]
                self.selected_vdc_inventory_df.loc[idx,self.rdly_selected_vdc] += num_cars
                ##
            else:
                lead_time_reward = 10 * num_cars / 4
        inventory_state = state[12:]
        current_inventory_state = inventory_state[0]
        overflow_cost = max(0, -current_inventory_state) * self.OVERFLOW_VAR_COST
        
        self.state, q = self.get_state()
        
        if self.steps == self.TRANING_STEPS:
            trans_reward += self.calc_transport_cost2(from_vdc,q)
        else:
            if action == 1:          
                trans_reward += self.calc_transport_cost2(from_vdc,to_vdc,q)
                ##.. update inventory DataFrame.
                link_info = self.Link_info[from_vdc][to_vdc]
                delivery_time = link_info["time"]
                dt = self.current_time + timedelta(hours = delivery_time) + timedelta(days = 1)
                delivered_time = dt.strftime("%Y-%m-%d %H:00:00")
                idx = self.selected_vdc_inventory_df.index[self.selected_vdc_inventory_df["arrival_time"]==delivered_time][0]
                self.selected_vdc_inventory_df.loc[idx,self.rdly_selected_vdc] += 20* q
            else:
                trans_reward = self.calc_transport_cost2(from_vdc,to_vdc,q) 
                self.num_of_send += q*20
                link_info = self.Link_info[from_vdc][to_vdc]
                delivery_time = link_info["time"]
                dt = self.current_time + timedelta(hours = delivery_time) + timedelta(days = 1)
                delivered_time = dt.strftime("%Y-%m-%d %H:00:00")
                idx = self.selected_vdc_inventory_df.index[self.selected_vdc_inventory_df["arrival_time"]==delivered_time][0]
                self.selected_vdc_inventory_df.loc[idx,self.rdly_selected_vdc] += 20* q

        inventory_state = state[12:]        
        current_inventory_state = inventory_state[0]
        overflow_cost = max(0, -current_inventory_state) * self.OVERFLOW_VAR_COST
        
        if self.steps==0:
            out = num_car_send_t0
        else:
            out = q*20 + num_car_send_t0
        ##update inventory DataFrame
        link_info2 = self.Link_info[from_vdc][to_vdc]
        delivery_time2 = link_info2["time"]
        dt2 = self.current_time + timedelta(hours = delivery_time2) + timedelta(days = 2) + timedelta(days = 1)
        delivered_time2 = dt2.strftime("%Y-%m-%d %H:00:00")
        
        idx = self.selected_vdc_inventory_df.index[self.selected_vdc_inventory_df["arrival_time"]==delivered_time2][0]
        self.selected_vdc_inventory_df.loc[idx,self.rdly_selected_vdc] -= out
        
        self.steps += 1
        if self.steps == self.TRANING_STEPS:
            trans_reward
            self.done = True
            
        self.current_time += timedelta(hours=self.DECISION_INTERVAL)
        
        return self.state, -sum([trans_reward, lead_time_reward, overflow_cost]), self.done, {
            'trans_reward' : trans_reward,
            'lead_time_reward' : lead_time_reward,
            'overflow_cost' : overflow_cost
        } 
        
    def calc_transport_cost(self, from_vdc, to_vdc, num_cars):
        if num_cars == 0:
            cost=2000
        else:
            num_r = num_cars // 20 + 1
            link_info = self.Link_info[from_vdc][to_vdc]
            distance = link_info['distance']
            cost = 2000*num_r + num_cars * distance * 3
        return cost

    def calc_transport_cost2(self, from_vdc, to_vdc, q):
        if q == 0:        
            cost=0
        else:
            link_info = self.Link_info[from_vdc][to_vdc]
            distance = link_info['distance']
            cost = 2000*q + q*20 * distance * 3
        return cost

    def reset(self):
        self.total_q=0
        self.q =0
        self.rail_available = True
        self.rdly_selected_vdc = "BE"
        self.me = "3A"
        self.num_of_send = 0
        self.df_inventory = copy.deepcopy(self.initial_data["df_inventory"])
        self.current_time = copy.deepcopy(self.initial_data["start_date"])
        self.selected_vdc_from_3A = self.df_from_3A[['arrival_time', self.rdly_selected_vdc]]
        self.selected_vdc_inventory_df = self.df_inventory[['arrival_time', self.rdly_selected_vdc]]
        self.next_vdc_capa= 200
        self.state, q = self.get_state()
        self.done = False
        self.steps = 0
        self.OVERFLOW_VAR_COST = 10

        return self.state
    
    def render(self, mode='human'):
        return None # Not implementing a nice graphic!


In [319]:

from collections import deque

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
    def __len__(self):
        return len(self.buffer)
    
def compute_td_loss(batch_size):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    state      = torch.autograd.Variable(torch.FloatTensor(state))
    next_state = torch.autograd.Variable(torch.FloatTensor(next_state))
    action     = torch.autograd.Variable(torch.LongTensor(action))
    reward     = torch.autograd.Variable(torch.FloatTensor(reward))
    done       = torch.autograd.Variable(torch.FloatTensor(done))
    q_values      = model(state)
    next_q_values = model(next_state)
    q_value = q_values.view(-1, env.action_space.n, 2).gather(2, action.unsqueeze(2)).squeeze(2)
    next_q_value = next_q_values.view(-1, env.action_space.n, 2).max(2)[0]
    expected_q_value = reward + gamma * next_q_value.sum(dim=1) * (1 - done)
    loss = (q_value.sum(dim=1) - torch.autograd.Variable(expected_q_value.data)).pow(2).mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [320]:
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)
env = HUFSEnv_v1("2011-01-03")
state = env.reset()
model = DQN(env.observation_space.shape[0], env.action_space.n)
optimizer = optim.Adam(model.parameters())

In [321]:
num_frames = 16000
batch_size = 160
replay_buffer = ReplayBuffer(100 * batch_size) 
gamma      = 0.99
losses = []
episode_reward = 0
avg_loss = 0
avg_losses = []
state = env.reset()
for frame_idx in range(1, num_frames + 1):
    if frame_idx %100==0:
        epsilon = epsilon_by_frame(frame_idx)
        action = get_action(env, state, model, epsilon)
        next_state, reward, done, _ = env.step(action)
        replay_buffer.push(state, action, reward, next_state, done)
        state = next_state
        episode_reward += reward
    if done:
        state = env.reset()
    if len(replay_buffer) > batch_size:
        loss = compute_td_loss(batch_size)
        losses.append(loss.item())
        avg_losses.append((avg_loss + loss.item())/frame_idx)
        avg_loss = 0
    try:
        avg_loss = avg_loss + loss.item()
    except:
        pass
env.close()

100


200


300


400


500


...

15600


15700


15800


15900


16000


In [312]:
torch.save(model.state_dict(), "model_files/INTERVER6_model_static_dic_256_1600_16_10.pt")

In [313]:
torch.save(model, "model_files/INTERVER6_model_256_1600_16_10.pt")

DQN(
  (layers): Sequential(
    (0): Linear(in_features=16, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2, bias=True)
  )
)

[2, 17, 31, 38, 41, 46, 56, 58, 198, 200, 200, 200, 197, 193, 198, 197]
200
0


0


0
False
-----------
action!! [0]
                           step!
[2, 17, 31, 38, 41, 46, 56, 58] [198, 200, 200, 200, 197, 193, 198, 197] False {'trans_reward': 132947319.14323357, 'lead_time_reward': 5.0, 'overflow_cost': 0}
------------
1
False
-----------
action!! [0]
                           step!


[17, 31, 38, 41, 46, 56, 58, 65] [200, 200, 200, 200, 195, 199, 200, 198] False {'trans_reward': 0, 'lead_time_reward': 5.0, 'overflow_cost': 0}
------------
2
False
-----------
action!! [0]
                           step!
[11, 18, 21, 26, 36, 38, 45, 52] [197, 198, 199, 200, 200, 197, 199, 198] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 42.5, 'overflow_cost': 0}
------------
3
False
-----------
action!! [0]
                           step!


[18, 21, 26, 36, 38, 45, 52, 63] [200, 199, 199, 197, 198, 199, 193, 194] False {'trans_reward': 0, 'lead_time_reward': 27.5, 'overflow_cost': 0}
------------
4
False
-----------
action!! [0]
                           step!
[1, 6, 16, 18, 25, 32, 43, 51] [199, 200, 198, 199, 197, 198, 199, 197] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 45.0, 'overflow_cost': 0}
------------
5
False
-----------
action!! [0]
                           step!


[6, 16, 18, 25, 32, 43, 51, 57] [193, 200, 198, 196, 200, 199, 197, 198] False {'trans_reward': 0, 'lead_time_reward': 2.5, 'overflow_cost': 0}
------------
6
False
-----------
action!! [0]
                           step!
[16, 18, 25, 32, 43, 51, 57, 66] [200, 198, 200, 200, 199, 197, 199, 194] False {'trans_reward': 0, 'lead_time_reward': 15.0, 'overflow_cost': 0}
------------
7
False
-----------
action!! [0]
                           step!


[18, 25, 32, 43, 51, 57, 66, 84] [199, 200, 198, 194, 200, 200, 200, 199] False {'trans_reward': 0, 'lead_time_reward': 40.0, 'overflow_cost': 0}
------------
8
False
-----------
action!! [0]
                           step!
[5, 12, 23, 31, 37, 46, 64, 74] [200, 199, 200, 198, 197, 196, 191, 198] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 45.0, 'overflow_cost': 0}
------------
9
False
-----------
action!! [0]
                           step!


...

[7, 15, 26, 41, 56, 61, 71, 94] [-21046, -20986, -20943, -20900, -20962, -20963, -20994, -20963] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 5.0, 'overflow_cost': 209530}
------------
156
False
-----------
action!! [0]
                           step!
[15, 26, 41, 56, 61, 71, 94, 114] [-21052, -20982, -20920, -20921, -20971, -20987, -20961, -20962] False {'trans_reward': 0, 'lead_time_reward': 17.5, 'overflow_cost': 209620}
------------
157
False
-----------
action!! [0]
                           step!


[6, 21, 36, 41, 51, 74, 94, 103] [-21047, -20962, -20922, -20945, -20961, -20986, -20944, -20948] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 37.5, 'overflow_cost': 209710}
------------
158
False
-----------
action!! [0]
                           step!
[1, 16, 21, 31, 54, 74, 83, 100] [-21002, -20941, -20901, -20942, -20974, -20963, -20962, -20946] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 15.0, 'overflow_cost': 209610}
------------
159
False
-----------
action!! [0]
                           step!


[16, 21, 31, 54, 74, 83, 100, 126] [-20980, -20926, -20923, -20967, -20981, -20993, -20961, -20961] False {'trans_reward': 0, 'lead_time_reward': 2.5, 'overflow_cost': 209740}
------------
160
False
-----------
action!! [0]
                           step!
[1, 11, 34, 54, 63, 80, 106, 118] [-20980, -20921, -20949, -20969, -20986, -20962, -20941, -20960] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 40.0, 'overflow_cost': 209810}
------------
161
False
-----------
action!! [0]
                           step!


[11, 34, 54, 63, 80, 106, 118, 128] [-20965, -20903, -20947, -20972, -20965, -20943, -20946, -20966] False {'trans_reward': 0, 'lead_time_reward': 2.5, 'overflow_cost': 209860}
------------
162
False
-----------
action!! [0]
                           step!
[14, 34, 43, 60, 86, 98, 108, 118] [-20941, -20928, -20961, -20962, -20989, -20961, -20965, -20987] False {'trans_reward': 126135.97641673015, 'lead_time_reward': 27.5, 'overflow_cost': 209650}
------------
163
False
-----------
action!! [0]
                           step!


KeyboardInterrupt: 

_


True

[15, 16, 19, 24, 29, 30, 36, 38, 41, 42, 43, 45, 0, 4, 6, 5, 5, 1, 7, 3, 1, 0, 3, 0] 868964.3349171111 False {'trans_reward': 868951.8349171111, 'lead_time_reward': 12.5}


TypeError: __init__() missing 1 required positional argument: 'start_date'

([0, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 array([0], dtype=uint8))

[9, 12, 15, 15, 17, 17, 17, 18, 19, 21, 22, 22, 3, 3, 4, 2, 9, 14, 8, 6, 10, 11, 9, 5]
[0] [12, 15, 15, 17, 17, 17, 18, 19, 21, 22, 22, 23, 4, 2, 9, 14, 8, 6, 10, 11, 9, 5, 4, 7] {'trans_reward': 2482719.528334603, 'lead_time_reward': 7.5}
[0] [15, 15, 17, 17, 17, 18, 19, 21, 22, 22, 23, 23, 9, 14, 8, 6, 10, 11, 9, 5, 4, 7, 10, 4] {'trans_reward': 0.0, 'lead_time_reward': 10.0}


[0] [15, 17, 17, 17, 18, 19, 21, 22, 22, 23, 23, 24, 8, 6, 10, 11, 9, 5, 4, 7, 10, 4, 2, 2] {'trans_reward': 2482719.528334603, 'lead_time_reward': 12.5}
[0] [17, 17, 17, 18, 19, 21, 22, 22, 23, 23, 24, 26, 10, 11, 9, 5, 4, 7, 10, 4, 2, 2, 0, 5] {'trans_reward': 0.0, 'lead_time_reward': 12.5}
[0] [17, 17, 18, 19, 21, 22, 22, 23, 23, 24, 26, 28, 9, 5, 4, 7, 10, 4, 2, 2, 0, 5, 2, 0] {'trans_reward': 2482719.528334603, 'lead_time_reward': 14.166666666666666}


[0] [17, 18, 19, 21, 22, 22, 23, 23, 24, 26, 28, 30, 4, 7, 10, 4, 2, 2, 0, 5, 2, 0, 5, 2] {'trans_reward': 0.0, 'lead_time_reward': 14.166666666666666}
[0] [18, 19, 21, 22, 22, 23, 23, 24, 26, 28, 30, 33, 10, 4, 2, 2, 0, 5, 2, 0, 5, 2, 5, 7] {'trans_reward': 2482719.528334603, 'lead_time_reward': 14.166666666666666}
[0] [19, 21, 22, 22, 23, 23, 24, 26, 28, 30, 33, 35, 2, 2, 0, 5, 2, 0, 5, 2, 5, 7, 1, 0] {'trans_reward': 0.0, 'lead_time_reward': 15.0}


[0] [1, 2, 2, 3, 3, 4, 6, 8, 10, 13, 15, 15, 0, 5, 2, 0, 5, 2, 5, 7, 1, 0, 2, 2] {'trans_reward': 2482719.528334603, 'lead_time_reward': 15.833333333333334}
[0] [2, 2, 3, 3, 4, 6, 8, 10, 13, 15, 15, 16, 2, 0, 5, 2, 5, 7, 1, 0, 2, 2, 5, 3] {'trans_reward': 124135.97641673015, 'lead_time_reward': 0.8333333333333334}
[0] [2, 3, 3, 4, 6, 8, 10, 13, 15, 15, 16, 16, 5, 2, 5, 7, 1, 0, 2, 2, 5, 3, 2, 4] {'trans_reward': 2482719.528334603, 'lead_time_reward': 1.6666666666666667}


[0] [3, 3, 4, 6, 8, 10, 13, 15, 15, 16, 16, 16, 5, 7, 1, 0, 2, 2, 5, 3, 2, 4, 2, 3] {'trans_reward': 124135.97641673015, 'lead_time_reward': 1.6666666666666667}
[0] [3, 4, 6, 8, 10, 13, 15, 15, 16, 16, 16, 16, 1, 0, 2, 2, 5, 3, 2, 4, 2, 3, 2, 3] {'trans_reward': 2482719.528334603, 'lead_time_reward': 2.5}
[0] [4, 6, 8, 10, 13, 15, 15, 16, 16, 16, 16, 17, 2, 2, 5, 3, 2, 4, 2, 3, 2, 3, 0, 2] {'trans_reward': 124135.97641673015, 'lead_time_reward': 2.5}
[0] [6, 8, 10, 13, 15, 15, 16, 16, 16, 16, 17, 17, 5, 3, 2, 4, 2, 3, 2, 3, 0, 2, 1, 3] {'trans_reward': 2482719.528334603, 'lead_time_reward': 3.3333333333333335}


...

[0] [10, 20, 24, 29, 36, 37, 46, 46, 51, 55, 61, 66, 4, 2, 2, 1, 5, 3, 5, 2, 5, 3, 4, 5] {'trans_reward': 372407.92925019044, 'lead_time_reward': 0.8333333333333334}
[0] [0, 4, 9, 16, 17, 26, 26, 31, 35, 41, 46, 52, 2, 1, 5, 3, 5, 2, 5, 3, 4, 5, 7, 7] {'trans_reward': 2730991.4811680634, 'lead_time_reward': 8.333333333333334}
[0] [4, 9, 16, 17, 26, 26, 31, 35, 41, 46, 52, 57, 5, 3, 5, 2, 5, 3, 4, 5, 7, 7, 12, 3] {'trans_reward': 496543.9056669206, 'lead_time_reward': 0.0}


[0] [9, 16, 17, 26, 26, 31, 35, 41, 46, 52, 57, 68, 5, 2, 5, 3, 4, 5, 7, 7, 12, 3, 8, 1] {'trans_reward': 2730991.4811680634, 'lead_time_reward': 3.3333333333333335}
[0] [16, 17, 26, 26, 31, 35, 41, 46, 52, 57, 68, 73, 5, 3, 4, 5, 7, 7, 12, 3, 8, 1, 3, 4] {'trans_reward': 496543.9056669206, 'lead_time_reward': 7.5}
[0] [17, 26, 26, 31, 35, 41, 46, 52, 57, 68, 73, 79, 4, 5, 7, 7, 12, 3, 8, 1, 3, 4, 2, 8] {'trans_reward': 2730991.4811680634, 'lead_time_reward': 13.333333333333334}


[0] [6, 6, 11, 15, 21, 26, 32, 37, 48, 53, 59, 65, 7, 7, 12, 3, 8, 1, 3, 4, 2, 8, 8, 0] {'trans_reward': 496543.9056669206, 'lead_time_reward': 14.166666666666666}
[0] [6, 11, 15, 21, 26, 32, 37, 48, 53, 59, 65, 67, 12, 3, 8, 1, 3, 4, 2, 8, 8, 0, 4, 0] {'trans_reward': 2855127.457584793, 'lead_time_reward': 5.0}
[0] [11, 15, 21, 26, 32, 37, 48, 53, 59, 65, 67, 70, 8, 1, 3, 4, 2, 8, 8, 0, 4, 0, 0, 1] {'trans_reward': 496543.9056669206, 'lead_time_reward': 5.0}


[0] [15, 21, 26, 32, 37, 48, 53, 59, 65, 67, 70, 73, 3, 4, 2, 8, 8, 0, 4, 0, 0, 1, 6, 1] {'trans_reward': 2855127.457584793, 'lead_time_reward': 9.166666666666666}
[0] [1, 6, 12, 17, 28, 33, 39, 45, 47, 50, 53, 53, 2, 8, 8, 0, 4, 0, 0, 1, 6, 1, 1, 3] {'trans_reward': 496543.9056669206, 'lead_time_reward': 12.5}


KeyboardInterrupt: 